In [111]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [112]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample.csv")

In [113]:
train.head()

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,0,0005019281,ADZPIG9QOCDG5,0,0,4.0,1203984000,good version of a classic,This is a charming version of the classic Dick...
1,1,0005019281,A35947ZP82G7JH,0,0,3.0,1388361600,Good but not as moving,It was good but not as emotionally moving as t...
2,2,0005019281,A3UORV8A9D5L2E,0,0,3.0,1388361600,Winkler's Performance was ok at best!,"Don't get me wrong, Winkler is a wonderful cha..."
3,3,0005019281,A1VKW06X1O2X7V,0,0,5.0,1202860800,It's an enjoyable twist on the classic story,Henry Winkler is very good in this twist on th...
4,4,0005019281,A3R27T4HADWFFJ,0,0,4.0,1387670400,Best Scrooge yet,This is one of the best Scrooge movies out. H...


In [114]:
train = train[['Score','Text']]

In [115]:
train.head()

,Score,Text
0,4.0,This is a charming version of the classic Dick...
1,3.0,It was good but not as emotionally moving as t...
2,3.0,"Don't get me wrong, Winkler is a wonderful cha..."
3,5.0,Henry Winkler is very good in this twist on th...
4,4.0,This is one of the best Scrooge movies out. H...


In [116]:
train = train.dropna()

In [117]:
train.head()

,Score,Text
0,4.0,This is a charming version of the classic Dick...
1,3.0,It was good but not as emotionally moving as t...
2,3.0,"Don't get me wrong, Winkler is a wonderful cha..."
3,5.0,Henry Winkler is very good in this twist on th...
4,4.0,This is one of the best Scrooge movies out. H...


In [118]:
test.head()

,Id,Score
0,5,NaN
1,11,NaN
2,17,NaN
3,46,NaN
4,47,NaN


In [119]:
import re
p = re.compile(r'[^\w\s]+')
train['Text'] = [p.sub('', x) for x in train['Text'].tolist()]

In [120]:
train['Text'] = train['Text'].str.lower().str.split()

In [121]:
train.head()

,Score,Text
0,4.0,"[this, is, a, charming, version, of, the, clas..."
1,3.0,"[it, was, good, but, not, as, emotionally, mov..."
2,3.0,"[dont, get, me, wrong, winkler, is, a, wonderf..."
3,5.0,"[henry, winkler, is, very, good, in, this, twi..."
4,4.0,"[this, is, one, of, the, best, scrooge, movies..."


In [122]:
stop = ['a','an','the','and','this','that']

In [123]:
train['Text'] = train['Text'].apply(lambda x: [item for item in x if item not in stop])

In [124]:
train.head()

,Score,Text
0,4.0,"[is, charming, version, of, classic, dickens, ..."
1,3.0,"[it, was, good, but, not, as, emotionally, mov..."
2,3.0,"[dont, get, me, wrong, winkler, is, wonderful,..."
3,5.0,"[henry, winkler, is, very, good, in, twist, on..."
4,4.0,"[is, one, of, best, scrooge, movies, out, henr..."


In [125]:
import copy
train1 = copy.deepcopy(train)

In [126]:
train1['Text'] = train1['Text'].apply(', '.join)

In [127]:
train1.head()

,Score,Text
0,4.0,"is, charming, version, of, classic, dickens, t..."
1,3.0,"it, was, good, but, not, as, emotionally, movi..."
2,3.0,"dont, get, me, wrong, winkler, is, wonderful, ..."
3,5.0,"henry, winkler, is, very, good, in, twist, on,..."
4,4.0,"is, one, of, best, scrooge, movies, out, henry..."


In [129]:
p1 = re.compile(r'[^\w\s]+')
train1['Text'] = [p1.sub('', x) for x in train1['Text'].tolist()]
train1['Text'] = train1['Text'].str.lower()

In [130]:
train1.head()

,Score,Text
0,4.0,is charming version of classic dickens tale he...
1,3.0,it was good but not as emotionally moving as c...
2,3.0,dont get me wrong winkler is wonderful charact...
3,5.0,henry winkler is very good in twist on classic...
4,4.0,is one of best scrooge movies out henry winkle...


In [131]:
X_train = train1['Text']
y_train = train1['Score']

In [132]:
X_train.head()

0    is charming version of classic dickens tale he...
1    it was good but not as emotionally moving as c...
2    dont get me wrong winkler is wonderful charact...
3    henry winkler is very good in twist on classic...
4    is one of best scrooge movies out henry winkle...
Name: Text, dtype: object

In [174]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC(dual=False,C = 0.2)),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train) 

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [172]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
text_clf2 = Pipeline([('tfidf', TfidfVectorizer()),
                     ('rdf',RandomForestClassifier()),
])
# Feed the training data through the pipeline
text_clf2.fit(X_train[:10000], y_train[:10000]) 

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [173]:
from sklearn.naive_bayes import MultinomialNB
text_clf3 = Pipeline([('tfidf', TfidfVectorizer()),
                     ('mnb',MultinomialNB()),
])
# Feed the training data through the pipeline
text_clf3.fit(X_train[:10000], y_train[:10000]) 

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...rue,
        vocabulary=None)), ('mnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [176]:
from sklearn.ensemble import VotingClassifier
#create a dictionary of our models
estimators=[('SVC',text_clf),('RDF',text_clf2),('MNB',text_clf3)]
#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators, voting='hard')

In [ ]:
#fit model to training data
ensemble.fit(X_train, y_train)

In [134]:
train_new = pd.read_csv("train.csv")

In [135]:
test_new = test.join(train_new, on='Id', how = 'inner',lsuffix='test_data', rsuffix='train_new')

In [136]:
test_new.head()

,Id,Idtest_data,Scoretest_data,Idtrain_new,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Scoretrain_new,Time,Summary,Text
0,5,5,NaN,5,0005019281,A2L0G56BNOTX6S,0,0,NaN,1383696000,Dickens updated.,This has been a favorite movie of mine for a l...
1,11,11,NaN,11,0005019281,A33EWPXESP9GQH,0,0,NaN,1390780800,Good Version,Even though i don't care for Henry Winklers a...
2,17,17,NaN,17,0005019281,A13KAQO9F5X0FN,0,0,NaN,1389657600,the fonz does scrooge,Anorher good movie for holiday watchers..a lit...
3,46,46,NaN,46,0005019281,A306NASGVUDFKF,10,14,NaN,1132963200,A refreshing twist on a Holiday classic,My wife and I grew up in New Hampshire where t...
4,47,47,NaN,47,0005019281,A38G1NN5SD81GD,0,1,NaN,1384905600,Not my favorite,"This is a first for me, I didn't like this mov..."


In [137]:
test_text = test_new[['Id','Text']]

In [138]:
test_text.head()

,Id,Text
0,5,This has been a favorite movie of mine for a l...
1,11,Even though i don't care for Henry Winklers a...
2,17,Anorher good movie for holiday watchers..a lit...
3,46,My wife and I grew up in New Hampshire where t...
4,47,"This is a first for me, I didn't like this mov..."


In [139]:
test_text = test_text.dropna()

In [140]:
p1 = re.compile(r'[^\w\s]+')
test_text['Text'] = [p1.sub('', x) for x in test_text['Text'].tolist()]

In [141]:
test_text.head()

,Id,Text
0,5,This has been a favorite movie of mine for a l...
1,11,Even though i dont care for Henry Winklers ac...
2,17,Anorher good movie for holiday watchersa littl...
3,46,My wife and I grew up in New Hampshire where t...
4,47,This is a first for me I didnt like this movie...


In [142]:
test_text['Text'] = test_text['Text'].str.lower().str.split()

In [143]:
test_text.head()

,Id,Text
0,5,"[this, has, been, a, favorite, movie, of, mine..."
1,11,"[even, though, i, dont, care, for, henry, wink..."
2,17,"[anorher, good, movie, for, holiday, watchersa..."
3,46,"[my, wife, and, i, grew, up, in, new, hampshir..."
4,47,"[this, is, a, first, for, me, i, didnt, like, ..."


In [144]:
test_text['Text'] = test_text['Text'].apply(', '.join)

In [145]:
test_text.head()

,Id,Text
0,5,"this, has, been, a, favorite, movie, of, mine,..."
1,11,"even, though, i, dont, care, for, henry, winkl..."
2,17,"anorher, good, movie, for, holiday, watchersa,..."
3,46,"my, wife, and, i, grew, up, in, new, hampshire..."
4,47,"this, is, a, first, for, me, i, didnt, like, t..."


In [146]:
p2 = re.compile(r'[^\w\s]+')
test_text['Text'] = [p2.sub('', x) for x in test_text['Text'].tolist()]
test_text['Text'] = test_text['Text'].str.lower()

In [147]:
predictions = text_clf.predict(test_text['Text'])

In [148]:
predictions.shape

(299991,)

In [149]:
#Create a  DataFrame with the passengers ids and our prediction regarding whether they survived or not
submission_2 = pd.DataFrame({'Id':test_text['Id'],'Score':predictions})
#Visualize the first 5 rows
submission_2.head()

,Id,Score
0,5,5.0
1,11,5.0
2,17,5.0
3,46,5.0
4,47,1.0


In [150]:
common = test_new.merge(submission_2,on=['Id'])
common=test_new[(~test_new['Id'].isin(common['Id']))]

In [151]:
common.shape

(9, 12)

In [152]:
common['Id'].values

array([ 106563,  245184,  864850, 1022346, 1082487, 1116131, 1243416,
       1253440, 1589648])

In [153]:
common.head(10)

,Id,Idtest_data,Scoretest_data,Idtrain_new,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Scoretrain_new,Time,Summary,Text
18795,106563,106563,NaN,106563,0790738074,A25VN8IR9DYJK3,4,5,NaN,1262390400,Amazing film,NaN
43189,245184,245184,NaN,245184,6301095294,A20QOIE60TAY2B,1,2,NaN,1254355200,Psychedelic Beatles,NaN
153210,864850,864850,NaN,864850,B0006B2R86,A61J00KQPWV0K,0,1,NaN,1338422400,They Call Me Trinity,NaN
180980,1022346,1022346,NaN,1022346,B000IU37T8,A3KEZLJ59C1JVH,17,20,NaN,1191024000,This DVD set has the POWER!,NaN
191684,1082487,1082487,NaN,1082487,B000RL5H6Y,A2W2ND26RMWZCA,1,1,NaN,1356825600,Watch Out For Fakes,NaN
197635,1116131,1116131,NaN,1116131,B000Y7U996,AGC19IYWC2KLA,1,4,NaN,1201564800,"Not terrible, but could have been a lot more",NaN
220075,1243416,1243416,NaN,1243416,B001VB8UAE,A144Z0HI5JJGH5,13,15,NaN,1243641600,How good do you have to be?,NaN
221862,1253440,1253440,NaN,1253440,B00275EHJG,A1H16TDWWGZEP5,0,3,NaN,1310601600,cheap!,NaN
281151,1589648,1589648,NaN,1589648,B009A87WU4,A5PZNLSCMM4KQ,4,5,NaN,1363478400,A Must-Have Collection,NaN


In [154]:
submission_2.head()

,Id,Score
0,5,5.0
1,11,5.0
2,17,5.0
3,46,5.0
4,47,1.0


In [155]:
common_id_score = common[['Id','Scoretest_data']]

In [156]:
common_id_score = common_id_score.rename(columns = {"Scoretest_data":"Score"})

In [157]:
common_id_score.head()

,Id,Score
18795,106563,NaN
43189,245184,NaN
153210,864850,NaN
180980,1022346,NaN
191684,1082487,NaN


In [158]:
submission_2_new = submission_2.append(common_id_score)

In [159]:
submission_2_new.tail()

,Id,Score
191684,1082487,NaN
197635,1116131,NaN
220075,1243416,NaN
221862,1253440,NaN
281151,1589648,NaN


In [160]:
submission_2_new['Score'].fillna((submission_2_new['Score'].mean()), inplace=True)

In [161]:
#Convert DataFrame to a csv file that can be uploaded
#This is saved in the same directory as your notebook
filename_2 = 'Yashvardhan_Das_6.csv'

submission_2_new.to_csv(filename_2,index=False)

print('Saved file: ' + filename_2)

Saved file: Yashvardhan_Das_6.csv
